# Generates a capacity.json file based on the roadnet file
#### <span style="color:red">Copy this notebook to the Simulation folder (needed for imported functions)</span>

In [1]:
import json
import numpy as np
from simulation_package.helper.Simulation_class import Simulation
from simulation_package.parameter_loader import load_parameters
from simulation_package.helper.initialization import initialize

In [2]:
road_network = "Manhattan"
ext_dict = {}
algorithm = "MP"
write_phase_to_json = False  
params = load_parameters(algorithm, road_network, ext_dict)
sim = Simulation(params, algorithm, write_phase_to_json)
initialize(sim, False)

2024-03-12 20:50:11,577	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


This cluster consists of
                1 nodes in total
                8.0 CPU resources in total
            


In [3]:
roadnet_file = "Simulation_Results/Manhattan/roadnet.json"
save_file = "capacities.json"

# Load the data from the JSON files
with open(roadnet_file) as f:
    data = json.load(f)

# THE NUMBER OF CARS ON EACH LANE SEEMS SMALL, BUT HAS BEEN COUNTED DIRECTLY DURING A SIMULATION IN THE FRONTEND/INDEX.HTML
# THE REASON IS THAT THE INTERSECTION HAS A WIDTH ITSELF (30 M IN THIS CASE) THAT REDUCES THE ACTUAL SPACE FOR VEHICLES ON LANE
conversion = {
    100: 6, # 100 meter --> max capacity for 6 cars 
    150: 12,
    270: 28,
    810: 89
    
}

capacities = {}

for road in data["roads"]:
    points = road["points"]
    delta_x = np.abs(points[0]["x"] - points[1]["x"])
    delta_y = np.abs(points[0]["y"] - points[1]["y"])
    
    difference = max(delta_x, delta_y)
    
    road_capacity = conversion[difference] + 12
    
    # find all lanes
    all_lanes = [lane for lane in sim.lanes_data if lane.startswith(road["id"])]
    
    for lane in all_lanes:
        capacities[lane] = road_capacity
        

# Open a file in write mode
with open(save_file, 'w') as f:
    # Write the dictionary to the file with an indent size of 4
    json.dump(capacities, f, indent=4)
    
    